# Training a CNN with Pytorch
We will now be using a expanded version of the digits data set to train a convolutional neural network.

We will be using the Pytorch library, which is one of the main deep learning libraries. In comparison to SKlearn, Pytorch provides much more control over the architecture of your neural network, allowing you to create your own types of layers.

## Import libraries, including PyTorch
If you are using your own computer, you will need to install Pytorch. Setup guidance is here: https://pytorch.org/get-started/locally/)

In [ ]:
import torch as torch
torch.set_num_threads(2)
import matplotlib.pyplot as plt
import torch.nn as nn

The next lines check if your computer has CUDA GPU capabilities and uses it if it can. Elab doesn't have available GPUs, this means that the model will still train, but it will be slower. More explanation below on what speeds you should be expecting.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Import some data
PyTorch has some built in datasets, which we will be using. We will be using the MNIST digits data set, which is a larger version of the sklearn digits dataset

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True,
    )
    
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor()
    )

## View basic data set information
Now that the dataset has been imported, you can view information about the train/test set using print(train_data) if you wish. You will see that there are 60000 training examples and 10000 test examples

In [ ]:
print(train_data)

Like the Sklearn task, let's plot one of the images to see what we're looking at. Note that these are grayscale images (i.e. different shades of grey) rather than full RGB colour images. The assignment data set will contain colour images.

In [ ]:
plt.imshow(train_data.data[1], cmap='Greys')
plt.show()

In [ ]:
train_data.data[1].size()


# Create the neural network architecture as a new class, CNN

Defining a 'class' is beyond the scope of the course here - for a thorough introduction, read some material on Object Oriented Programming. Briefly, we can think of a 'class' as a grouping for a set of related variables and functions.

The CNN class below creates the neural network architecture. Note that nn.sequential means that layers are added sequentially, so conv1 consists of a conv2d layer, a ReLU activation and a max pooling layer. Each layer type has its own parameters (in_channels, out_channels, etc.)

In conv1, the convolution (conv2d) layer, we create 16 convolution filters (out_channels), a free choice by us (it can be any number). 

In the 2nd set of convolutions, defined by *conv2*, we have 16 convolutions from *conv1*, so our first parameter is 16 and we choose to have 32 output filters. Note that, for *conv2*, the parameters have been written in shorthand, without explicitly referring to the parameter variable name.

The function, forward, describes how the whole network is put together from its component layers. It shows how we first put our input through *conv1*, then *conv2*, then *out* (a fully connected layer). 

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv3 = nn.Sequential(         
            nn.Conv2d(32, 64, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        #self.out = nn.Linear(32 * 7 * 7, 10)
        self.out = nn.Linear(64 * 3 * 3, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization

## Instantiate the neural network object
The class code above describes what our CNN object looks like, but we need to actual create an instance of the object. We do a similar thing when we train models in sklearn

In [ ]:
cnn = CNN() # note that the variable name can be anything here, but I've chosen cnn for clarity

# Define the loss function and optimizer 
We now define the loss function and optimizer to train the network. Unlike standard stochastic gradient descent (what we looked at in the lectures), Adam is a variant that uses an adaptive learning rate to allow for faster convergence in many situations.

We set the initial learning rate at 0.01. This is quite large and may lead to non-convergence in other problems, but in this case I know it works (through trial and error), and it means that we will be able to see the impact of training in relatively few iterations

We use the cross entropy loss, which is the go-to for classification problems, rather than something like the mean squared error.

In [ ]:
loss_func = nn.CrossEntropyLoss()   
from torch import optim
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)

# Set up training parameters

Before attempting to train, we use the dataloader utility in pytorch to set up how training will work: 
the *batch_size* sets the mini-batch used in SGD (i.e. how many examples are considered at each step - in the lectures, we consider one example at a time, so-called online SGD). *shuffle* shuffles the data at each epoch, to reduce the likelihood of a weird convergence. This might happen, for instance, if all of the digits are in order, so that all of the 1s are trained first. This could lead to a situation where the network thinks that it has finished training because it sees thousands of examples of 1s, and only trains to recognise 1s. *num_workers* relates to how the data is handled in the memory (e.g. whether there are multiple sub-processes)

In [ ]:
from torch.utils.data import DataLoader
loaders = {
    'train' : DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
}

## Training function
We now define how model training works, over num_epochs. Note how the loss is computed. Gradients are computed using loss.backward, and then weights are updated using optimizer.step()

In [ ]:
num_epochs = 3
def train(num_epochs, cnn, loaders):
    
    # this sets the model mode - (i.e. layers like dropout, batchnorm etc behave differently during training compared to testing)
    # note that this function was not defined explicitly in CNN, but because CNN is a type of nn.Module, it inherits some functions
    # from the more general nn class.
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
       
            b_x = images
            b_y = labels
            
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
            pass     
        pass
    pass

After you have run everything above, call the training function below.

**if you are running this on elab, note that it will take a *long* time to run. You may get faster results by running this on a local machine (it takese about 10 seconds on my laptop...)**

In [ ]:
train(num_epochs, cnn, loaders)

In a standard gradient descent, you would expect the training loss to go down consistently. However, we are estimating the gradient each time on a subset of the data. This means that training loss can go up as well as down - however, over enough time, it can be shown that it will converge. One of the big advantages is that SGD requires a lot less memory, as we deal with only a small number of training examples at a time.

# Test the model

Now that the model has trained, let's see how well it does...

Note that the CNN model does not have a sigmoid (or softmax) activation layer - this means that the outputs are not bound between 0 and 1. If we wanted the *probability* of each class, we would need to add a softmax layer to the output. In this case, we just care about the most likely class, so we can just select the index that corresponds to the highest number.

In [ ]:
def test():
    # Test the model
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders['test']:
            test_output, last_layer = cnn(images) ## this runs the trained cnn. Note from the cnn class, test_output gives the model output (10 x 1 values), and last_layer gives the inputs into the last layer
            ## torch.max finds the highest value of test_output. the [0] array element returns the maximum value, the [1] element
            ## gives the index of that element. squeeze reshapes the data from a nx1 array into a list
            pred_y = torch.max(test_output, 1)[1].data.squeeze() 
            #print(pred_y)
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
        pass
        print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    
    pass
test()